# Úvod do Kerasu

Keras je knihovna pro rychlé prototypování neuronových sítí postavená na knihovnách s automatickým výpočtem gradientů.
- Frontend nabízí API pro prototypování, různé typy vrstvev, optimalizačních kritérií, ... Každá vrstva definuje pouze dopředný průchod, který sestává ze standardních atomických operací jako je součet, násobení, umocňování apod.
- Backend pak každou z těchto operací implementuje a zároveň definuje jejich zpětný průchod, tj. gradient na jednotlivé operandy.

Po navržení grafu neuronové sítě poskládáním vrstev za sebe se model zkompiluje, čímž se definuje celkový dopředný průchod od vstupu až po vrchol, a zároveň zřetězením a optimalizací atomických operací, ze kterých je tvořen, se definuje i zpětný průchod. Takto je docíleno automatického výpočtu gradientů a uživatel proto vůbec nemusí řešit, zda je jeho kód pro zpětný průchod správně, efektivní apod. - knihovna vše zajistí sama.

V současné době je výchozím backendem Kerasu knihovna tensorflow, která zároveň umožňuje operace provádět na GPU a tím výpočty výrazně urychlit. Keras však podporuje více backendů, např. Theano, na kterém kdysi začínal.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from IPython.core.debugger import set_trace

plt.rcParams['figure.figsize'] = (12., 8.)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## Načtení dat

Keras integruje nejpopulárnější testovací datasety přímo ve svém API. CIFAR-10 je jedním z nich. Načtení dat je tak velmi jednoduché:

In [ ]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print(X_train.shape, X_train.dtype)
print(y_train.shape, y_train.dtype)
print(X_test.shape, X_test.dtype)
print(y_test.shape, y_test.dtype)

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_train, num_test = X_train.shape[0], X_test.shape[0]
x_dim = X_train.shape[1] * X_train.shape[2] * X_train.shape[3]

Rozdílem oproti manuálnímu načítání z minula je, že anotace `y` jsou sloupcové vektory (matice Nx1). Navíc `y_train` je datového typu `uint8`, místo `int` či `int64`.

In [ ]:
for i, cls in enumerate(classes):
    cls_ids, = np.where(y_train[:, 0] == i)
    draw_ids = np.random.choice(cls_ids, size=10)
    
    for j, k in enumerate(draw_ids):
        plt.subplot(10, 10, j * 10 + i + 1)
        plt.imshow(X_train[k])
        plt.axis('off')
        if j == 0:
            plt.title(cls)
plt.show()

### Preprocessing

Nejjednodušším způsobem trénování v Kerasu je volání `sklearn`-like funkce `fit`, která chce jako argument celá trénovací data v podobě matice `X` a vektoru labelů `y`. Jelikož dovnitř nevidíme a nemůžeme zajistit volání preprocessingu pro každý vzorek, trénovací data tentokrát předzpracujeme celá data najednou a pouze jednou, nikoliv "online" jako minule.

In [ ]:
def preprocess(rgb_batch, resize=None):
    if isinstance(resize, tuple):
        rgb_batch = [cv2.resize(rgb, (32, 32)) for rgb in rgb_batch]
    X = np.array(rgb_batch, dtype=np.float32) / 255.
    m = np.mean(X, axis=(1, 2))
    X -= m[:, None, None, :]
    return X

In [ ]:
X_train_mat = preprocess(X_train).reshape(num_train, x_dim)
X_test_mat = preprocess(X_test).reshape(num_test, x_dim)

Pro trénování multiclass logistické regrese se softmaxem Keras vyžaduje, aby anotace `y` byla skutečně v one-hot repreztentaci. Převedení lze provést velmi jednoduše pomocnou funkcí:

In [ ]:
from keras import utils

In [ ]:
y_train_mat = utils.to_categorical(y_train, len(classes))
y_test_mat = utils.to_categorical(y_test, len(classes))

# print prvnich 5 labelu
print('jako vektor (trida = int):\n', y_train[:5])
print('jako matice (one-hot):\n', y_train_mat[:5, :])

## Keras: sekvenční API

Keras nabízí dva způsoby definování modelů: sekvenční a funkcionální. Sekvenční je původní, starší způsob, který je velmi jednoduchý, ale je omezen na dopředné sítě. Pokud např. některá vrstva přijímá vstup z více zdrojů, modeluje se takovýto graf v sekvenčním paradigma velmi komplikovaně. Proto bylo zavedeno funkcionální API, které je jen o něco málo "ukecanější", zato mnohem flexibilnější. Ukážeme si obě varianty, přičemž začneme sekvenční.

In [ ]:
from keras.layers import Dense
from keras.models import Sequential

Základní třídou reprezentující neuronovou síť je `Sequential`. Lineární (afinní) vrstva se pak v Kerasu nazývá `Dense`. Pro ukázku si zadefinujeme model ekvivalentní našemu `TwoLayerPerceptron` z minulého cvičení.

In [ ]:
feed_forward = Sequential()
feed_forward.add(Dense(200, input_shape=(x_dim,), activation='relu'))
feed_forward.add(Dense(len(classes), activation='softmax'))

`Dense(200, ...)` znamená, že vrstva bude mít výstup o rozměru 200, přičemž velikost vstupu se převezme z velikosti výstupu předchozí vrstvy. Keras zároveň kombinuje vrstvy s aktivacemi, takže ty pak není nutné zadávat samostatně.

Všimněme si také, že poslední vrstva narozdíl od minula obsahuje softmax. Je to kvůli cross entropy lossu, který defaultně v Kerasu vyžaduje, aby výstup byly pravděpodobnosti, avšak lze to jednoduše změnit.

Model má následující strukturu:

In [ ]:
feed_forward.summary()

## Keras: funkcionální API

Vyzkoušíme si také funkcionální API, které se liší pouze ve způsobu definice modelu. Úplně stejný model lze funkcionálním API vytvořit následovně:

In [ ]:
from keras.models import Input, Model

In [ ]:
x = Input(shape=(x_dim,))
h = Dense(200, activation='relu')(x)
p = Dense(len(classes), activation='softmax')(h)
feed_forward = Model(inputs=x, outputs=p)

In [ ]:
feed_forward.summary()

Výhodou tohoto přístupu je mnohem větší flexibilita. Pokud by totiž např. `p` záviselo zároveň na `x`, jak je tomu např. u residuálních sítí, stačilo by zadat `p = Residual(params)(x, h)`. U sekvenčního modelu metoda `add` vždy předpokládá, že vstup do následující vrstvy je výstup té předchozí.

Všimněme si, že vrstvy v Kerasu mají přetížený operátor volání `()`, a tedy např. `h = Dense(...)(x)` je prostě zavolání "funkce" se vstupem `x`. `x` je ale pouze objekt, který reprezentuje vstup, aniž by ale měl nějaký konkrétní obrázek či vektor přiřazený. Proměnná `h` je tedy pouze *symbolická* reprezentace lineární vrstvy - definice funkce, která očekává vektor o rozměru `x_dim` a vrací vektor s rozměrem `200`. Podobně pak `model` je složeninou jednotlivých vrstev a tedy opět pouze symbolická reprezentace, tj. definice funkce. Skutčené hodnoty výstupu `p` budeme znát až po zavolání s konkrétním vstupem (obrázkem).

Zbytek jako kompilace, trénování apod. je úplně stejný, můžeme tedy znovu projet kód nahoře, tentokrát s nově definovaným modelem a dostaneme stejné výsledky.

Zvolíme metodu optimalizace: prozatím stejnou jako minule, tj. stochastic gradient descent.

In [ ]:
from keras import optimizers

In [ ]:
opt = optimizers.SGD(lr=0.01)

Keras funguje na statickém, tzv. define-and-run modelu. To znamená, že nejprve zadefinujeme strukturu modelu, kterou poté zkompilujeme a během trénování se síť nemůže nijak měnit. Tímto krokem se ze sítě stane "binární blob", který bere vstup a vrací výstupní pravděpodobnosti.

In [ ]:
feed_forward.compile(opt, 'categorical_crossentropy', metrics=['accuracy'])

### Inicializace

Inicializace probíhá při definování vrstvy, v tuto chvíli už mají všechny parametry výchozí hodnoty (váhy náhodné, biasy nuly)

In [ ]:
feed_forward.layers[-1].get_weights()

Jediná inicializace tedy bude vynulovat pole, které bude ukládat historii hodnot lossu a accuracy.

In [ ]:
ffw_history = []

### Trénování 

Jak bylo řečeno, trénování zajišťuje metoda `fit`, které předáme trénovací data a další parametry jako velikost dávky (batch_size), počet epoch, validační data a další.

In [ ]:
batch_size = 20
epochs = 20
valid_data = X_test_mat, y_test_mat
# opt.lr.assign(0.001)

h = feed_forward.fit(X_train_mat, y_train_mat, batch_size=batch_size, epochs=epochs,
                     validation_data=valid_data, shuffle=True)
ffw_history.append(h)

In [ ]:
def plot_keras_history(histories):
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    
    plt.figure(figsize=(10, 3))
    for mi, metric in enumerate(('loss', 'acc')):
        plt.subplot(1, 2, mi + 1)
        for si, subset in enumerate(('', 'val_')):
            y = sum([h.history[subset + metric] for h in histories], [])
            plt.plot(y, color=colors[si])
        plt.xlabel('epoch')
        plt.ylabel(metric)
    
    plt.show()

In [ ]:
plot_keras_history(ffw_history)

### Validace

In [ ]:
feed_forward.evaluate(x=X_test_mat, y=y_test_mat)

### Predikce

Pokud máme natrénovaný model, můžeme predikovat třídu neznámého obrázku metodou `predict`.

In [ ]:
from urllib.request import urlopen
from skimage import io

In [ ]:
url = 'https://camo.githubusercontent.com/7e8b7ea66e6dbc2fbcd72bc2a105ed464de1b6b1/687474703a2f2f6661726d352e737461746963666c69636b722e636f6d2f343037302f343731373336333934355f623733616664373861392e6a7067'
rgb_test = io.imread(url)
plt.imshow(rgb_test)
plt.show()

In [ ]:
def predict_and_show(rgb_test, model):
    if len(rgb_test.shape) < 4:
        rgb_test = rgb_test.reshape(1, *rgb_test.shape[-3:])
    xt = preprocess(rgb_test, resize=model.input_shape[1:3])
    xt = xt.reshape(xt.shape[:1] + model.input_shape[1:])
    pt = model.predict(xt).ravel()
        
    plt.figure(figsize=(5, 5))
    plt.imshow(rgb_test[0, ...])
    ids = np.argsort(-pt)
    for i, ci in enumerate(ids):
        plt.gcf().text(1., 0.8 - 0.075 * i, '{}: {:.2f} %'.format(classes[ci], 100. * pt[ci]), fontsize=24)
    plt.subplots_adjust()
    plt.show()

In [ ]:
predict_and_show(rgb_test, feed_forward)